Benjamin Bierlein 76418644 Econ 125 Spring 2025 HW 7

9.22

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# Load and prepare data
df = pd.read_csv("phillips5_aus.csv")
df.rename(columns={'inf': 'infl'}, inplace=True)
df['infl_lag1'] = df['infl'].shift(1)
df['infl_lag4'] = df['infl'].shift(4)
data = df.dropna(subset=['infl', 'infl_lag1', 'infl_lag4', 'du'])

# Fit model
model = smf.ols('infl ~ infl_lag1 + infl_lag4 + du', data=data).fit(cov_type='HAC', cov_kwds={'maxlags': 4})

# Forecast inflation for 2016Q2–Q4
forecast_values = []
conf_intervals = []
last_obs = data.iloc[-1]

print("Forecasted Inflation Rates with 95% Confidence Intervals:")
for q in ['2016Q2', '2016Q3', '2016Q4']:
    X_pred = pd.DataFrame([{
        'infl_lag1': last_obs['infl'],
        'infl_lag4': data.iloc[-4]['infl'],
        'du': 0
    }])
    pred = model.get_prediction(X_pred)
    mean = pred.predicted_mean[0]
    ci = pred.conf_int(alpha=0.05)[0]
    print(f"{q}: Forecast = {mean:.4f}, 95% CI = [{ci[0]:.4f}, {ci[1]:.4f}]")

    new_row = {'infl': mean, 'infl_lag1': last_obs['infl'], 'infl_lag4': data.iloc[-4]['infl'], 'du': 0}
    data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
    last_obs = new_row




Forecasted Inflation Rates with 95% Confidence Intervals:
2016Q2: Forecast = 0.3718, 95% CI = [0.1665, 0.5771]
2016Q3: Forecast = 0.4837, 95% CI = [0.3864, 0.5810]
2016Q4: Forecast = 0.4825, 95% CI = [0.3967, 0.5684]


9.25

In [ ]:
# Load and prepare data
df = pd.read_csv("cons_inc.csv")
df['DC'] = df['cons'].diff()
df['DY'] = df['y'].diff()
df['DC_lag1'] = df['DC'].shift(1)
df['DY_lag3'] = df['DY'].shift(3)
df = df.dropna(subset=['DC', 'DY', 'DC_lag1', 'DY_lag3'])

# Estimate both models
model1 = smf.ols("DC ~ DC_lag1 + DY", data=df).fit()
model2 = smf.ols("DC ~ DY + DY_lag3", data=df).fit()

# Forecast consumption for 2016Q1–Q3
last_index = df.index[-4]
future_df = df.iloc[-3:]
prev_dc = df.loc[last_index, 'DC']
prev_c = df.loc[last_index, 'cons']

# Model 1: Recursive
model1_forecasts = []
for i in range(3):
    dc_pred = model1.params['Intercept'] + model1.params['DC_lag1'] * prev_dc + model1.params['DY'] * future_df.iloc[i]['DY']
    prev_dc = dc_pred
    prev_c += dc_pred
    model1_forecasts.append(round(prev_c, 0))

# Model 2: Non-recursive
prev_c = df.loc[last_index, 'cons']
model2_forecasts = []
for i in range(3):
    dc_pred = model2.params['Intercept'] + model2.params['DY'] * future_df.iloc[i]['DY'] + model2.params['DY_lag3'] * future_df.iloc[i]['DY_lag3']
    prev_c += dc_pred
    model2_forecasts.append(round(prev_c, 0))

# Compare to actual consumption
actual_c = df.iloc[-3:]['cons'].values
msfe1 = np.mean((np.array(model1_forecasts) - actual_c)**2)
msfe2 = np.mean((np.array(model2_forecasts) - actual_c)**2)

# Print results
print("\nForecasted Consumption (Model 1):")
for i, q in enumerate(["2016Q1", "2016Q2", "2016Q3"]):
    print(f"{q}: Forecast = {model1_forecasts[i]:,.0f}, Actual = {actual_c[i]:,.0f}")

print("\nForecasted Consumption (Model 2):")
for i, q in enumerate(["2016Q1", "2016Q2", "2016Q3"]):
    print(f"{q}: Forecast = {model2_forecasts[i]:,.0f}, Actual = {actual_c[i]:,.0f}")

print(f"\nMSFE for Model 1: {msfe1:.2f}")
print(f"MSFE for Model 2: {msfe2:.2f}")


Forecasted Consumption (Model 1):
2016Q1: Forecast = 236,395, Actual = 236,523
2016Q2: Forecast = 237,625, Actual = 237,613
2016Q3: Forecast = 238,793, Actual = 238,664

Forecasted Consumption (Model 2):
2016Q1: Forecast = 235,913, Actual = 236,523
2016Q2: Forecast = 236,716, Actual = 237,613
2016Q3: Forecast = 237,685, Actual = 238,664

MSFE for Model 1: 11056.33
MSFE for Model 2: 711716.67


12.7

In [ ]:
# Load macro data
df = pd.read_csv("usmacro.csv")

# Run ADF tests
adf_u = adfuller(df['u'].dropna(), maxlag=1, regression='c', autolag=None)
adf_g = adfuller(df['g'].dropna(), maxlag=0, regression='c', autolag=None)
adf_inf = adfuller(df['inf'].dropna(), maxlag=2, regression='c', autolag=None)

# Print ADF results
print("\nADF Test Results (using constant only):")
print(f"Unemployment: ADF = {adf_u[0]:.4f}, p = {adf_u[1]:.5f}, lags = {adf_u[2]} → {'Stationary' if adf_u[1] < 0.05 else 'Non-stationary'}")
print(f"GDP Growth:   ADF = {adf_g[0]:.4f}, p = {adf_g[1]:.5f}, lags = {adf_g[2]} → {'Stationary' if adf_g[1] < 0.05 else 'Non-stationary'}")
print(f"Inflation:    ADF = {adf_inf[0]:.4f}, p = {adf_inf[1]:.5f}, lags = {adf_inf[2]} → {'Stationary' if adf_inf[1] < 0.05 else 'Non-stationary'}")


ADF Test Results (using constant only):
Unemployment: ADF = -4.4759, p = 0.00022, lags = 1 → Stationary
GDP Growth:   ADF = -9.3452, p = 0.00000, lags = 0 → Stationary
Inflation:    ADF = -3.9861, p = 0.00148, lags = 2 → Stationary
